In [1]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import pickle
import random

Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Europe + NorthAmerica + SouthAmerica + Oceania
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

w_length = 200

### Consolidate and pickle country files

In [2]:
for country in countriesOfInterest:
    if len(glob.glob("Raw Track Data\\" + country + ".p")) == 0 :
        hold = pd.DataFrame()
    else:
        hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    print(country, hold.shape)
    countryTracks = glob.glob("Raw Track Data\\" + country + "*.csv")
    print(country, len(countryTracks))
    for file in countryTracks:
        new = pd.read_csv(file)
        new["Country"] = file[15:17]
        new["Year"] = file[18:22]
        new["Playlist"] = file[23:-4]
        hold = hold.append(new)
        pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )
        os.remove(file)
    hold = hold.drop_duplicates(["track_id", "start"])
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

DZ (20649, 34)
DZ 34
EG (482554, 34)
EG 32
MA (27489, 34)
MA 34
ZA (1944432, 34)
ZA 25
TN (64601, 34)
TN 32
BH (77907, 34)
BH 33
HK (661767, 34)
HK 29
IN (373346, 34)
IN 22
ID (1458006, 34)
ID 21
IL (1144769, 34)
IL 23
JP (1547009, 34)
JP 15
JO (60159, 34)
JO 29
KW (34879, 34)
KW 33
LB (12584, 34)
LB 29
MY (632297, 34)
MY 27
OM (186615, 34)
OM 34
PS (70584, 34)
PS 33
PH (720795, 34)
PH 33
QA (1643, 34)
QA 49
SA (0, 34)
SA 49
SG (370111, 34)
SG 36
TW (2015606, 34)
TW 40
TH (1243907, 34)
TH 40
AE (317609, 34)
AE 48
VN (1221942, 34)
VN 37
AD (24354, 34)
AD 33
AT (423551, 34)
AT 34
BE (630688, 34)
BE 33
BG (7218, 34)
BG 33
CY (95846, 34)
CY 25
CZ (75704, 34)
CZ 20
DK (2358831, 34)
DK 25
EE (2386, 34)
EE 22
FI (1740323, 34)
FI 23
FR (1624294, 34)
FR 19
DE (706830, 34)
DE 24
GR (0, 34)
GR 22
HU (4368, 34)
HU 23
IS (264150, 34)
IS 20
IE (1261504, 34)
IE 20
IT (938928, 34)
IT 19
LV (1966, 34)
LV 22
LI (0, 34)
LI 22
LT (0, 34)
LT 22
LU (1402, 34)
LU 22
MT (0, 34)
MT 22
MC (5782, 34)
MC 22
NL (1

In [3]:
hold

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,p1,p2,p3,p4,...,t7,t8,t9,t10,t11,t12,track_id,Country,Year,Playlist
681,0.00000,0.49914,0.000,-60.000,0.00000,-60.000,0.019,0.040,0.045,0.024,...,14.833,5.359,-27.228,0.973,-10.640,-7.228,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
682,0.49914,0.22644,1.000,-60.000,0.03370,-10.164,0.646,0.420,0.522,0.560,...,-20.233,-14.167,-6.882,34.476,41.585,7.220,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
683,0.72558,0.12181,0.381,-31.803,0.00782,-26.013,0.940,0.858,0.439,0.460,...,16.783,-18.108,13.722,32.195,-23.695,-3.539,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
684,0.84739,0.60340,0.952,-26.892,0.18692,-14.485,0.073,0.149,0.097,0.237,...,64.849,-42.428,21.399,50.362,-43.406,-18.990,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
685,1.45079,0.20980,0.775,-21.574,0.04262,-13.192,0.296,0.377,0.302,0.450,...,38.661,-2.748,52.698,13.987,-17.991,-5.808,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47787,427.38063,0.32308,0.121,-28.541,0.01170,-25.864,0.005,0.025,0.003,0.003,...,-19.117,-0.909,17.050,15.046,-8.665,13.584,spotify:track:4D3nttJPU6L0M2epr7sId6,NZ,2010,37i9dQZF1DX4DlNbZckE1g
47788,427.70372,0.45583,1.000,-32.546,0.05982,-15.182,0.005,0.118,0.011,0.004,...,-3.051,-10.341,1.615,-7.558,-29.854,2.496,spotify:track:4D3nttJPU6L0M2epr7sId6,NZ,2010,37i9dQZF1DX4DlNbZckE1g
47789,428.15955,2.65796,0.958,-23.281,0.09433,-10.394,0.054,1.000,0.035,0.018,...,-52.148,-3.183,-6.242,-17.094,-75.283,-18.306,spotify:track:4D3nttJPU6L0M2epr7sId6,NZ,2010,37i9dQZF1DX4DlNbZckE1g
47790,430.81751,0.78363,0.199,-42.590,0.05891,-40.270,0.048,1.000,0.049,0.036,...,-63.283,28.142,19.375,-15.269,-12.209,-3.552,spotify:track:4D3nttJPU6L0M2epr7sId6,NZ,2010,37i9dQZF1DX4DlNbZckE1g


### Remove non-unique tracks

In [4]:
allTracks = pd.DataFrame()
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[["Country", "Year", "track_id", "Playlist"]]
    allTracks = allTracks.append(hold)
allTracks

,Country,Year,track_id,Playlist
19557,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19558,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19559,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19560,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19561,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
...,...,...,...,...
47787,NZ,2010,spotify:track:4D3nttJPU6L0M2epr7sId6,37i9dQZF1DX4DlNbZckE1g
47788,NZ,2010,spotify:track:4D3nttJPU6L0M2epr7sId6,37i9dQZF1DX4DlNbZckE1g
47789,NZ,2010,spotify:track:4D3nttJPU6L0M2epr7sId6,37i9dQZF1DX4DlNbZckE1g
47790,NZ,2010,spotify:track:4D3nttJPU6L0M2epr7sId6,37i9dQZF1DX4DlNbZckE1g


In [5]:
hold = allTracks.groupby("track_id").nunique()
print(hold.shape[0], "Unique tracks per country")
keep = hold[hold.Country==1].index
allTracks = allTracks[allTracks.track_id.isin(keep)]
print(len(keep), "Unique tracks across countries")

78734 Unique tracks per country
58326 Unique tracks across countries


In [6]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[hold.track_id.isin(keep)]
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

### Create training, testing, and validation datasets

In [7]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    test = hold.loc[hold.track_id.isin(testTracks)]
    hold = hold.loc[~hold.track_id.isin(testTracks)]
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    val = hold.loc[hold.track_id.isin(testTracks)]
    train = hold.loc[~hold.track_id.isin(testTracks)]
    pickle.dump( train, open( "Raw Track Data\\" + country + "_train.p", "wb" ) )
    pickle.dump( test, open( "Raw Track Data\\" + country + "_test.p", "wb" ) )
    pickle.dump( val, open( "Raw Track Data\\" + country + "_val.p", "wb" ) )

In [8]:
countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
for country in countriesOfInterest:
    train = pickle.load( open( "Raw Track Data\\" + country + "_train.p", "rb" ) )
    test = pickle.load( open( "Raw Track Data\\" + country + "_test.p", "rb" ) )
    val = pickle.load( open( "Raw Track Data\\" + country + "_val.p", "rb" ) )
    print(country, "train", len(pd.unique(train.track_id)), 
          "\t test",len(pd.unique(test.track_id)) , 
          "\t val", len(pd.unique(val.track_id)))

HK train 743 	 test 232 	 val 185
JP train 1284 	 test 401 	 val 321
ZA train 1058 	 test 330 	 val 264
TN train 90 	 test 28 	 val 22
TR train 1288 	 test 402 	 val 321
GB train 682 	 test 212 	 val 170
MX train 826 	 test 257 	 val 206
US train 1365 	 test 426 	 val 341
CO train 1126 	 test 351 	 val 281
EC train 356 	 test 110 	 val 88
AU train 756 	 test 236 	 val 189
NZ train 420 	 test 131 	 val 105
